In [11]:
##Classify image in image_path

#Function that labels an image using an existing tensorflow model:
##Inputs:
###image_path - filepath to an image to label. File type should be '.jpg'
###model_graph_path - filepath to a tensorflow model graph to use for classification. File type should be '.pb'
###model_labels_path - filepath to a text file of label names generated when training a tensorflow model. File type should be '.txt'
##Outputs:
###Array with first row corresponding to class likelihood and second row corresponding to class text labels
def classify_image(image_path, model_graph_path, model_labels_path):
    
    import tensorflow as tf, sys
    
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()
    
    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line 
                       in tf.gfile.GFile(model_labels_path)]
    
    # Unpersists graph from file
    with tf.gfile.FastGFile(model_graph_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
    
    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})
    
    #EricW help on proper format of this output matrix. Would ideally like labeled vector or equivalent that can be coerced into a matrix
    predictionsout = [predictions[0], label_lines]
    
    return predictionsout

In [12]:
#Example usage of function above

predictiontest = classify_image('/opt/notebooks/PassFailHoldout/skiveandscratch2.jpg', '/opt/notebooks/output_graph.pb', '/opt/notebooks/output_labels.txt')